In [5]:
import pyvisa as visa
import sounddevice as sd
import numpy as np
from time import sleep
import wave as wav

In [2]:
t_freq = 1    # Time per frequency
t_pause = 0.1  # Pause between two frequencies
n_freq = 20   # Number of different frequencies
f_min = 500    # Minimum frequency
f_max = 20e3  # Maximum frequency

f_c = 40e3    # Carrier frequency

fs = 300e3    # Samplingrate
vf = np.linspace(f_min, f_max, n_freq)


# Configure Waveform Generator

In [136]:
def enableAWG(inst, channel: int = 1):
  inst.write(f"OUTP{channel} ON")

def disableAWG(inst, channel: int = 1):
  inst.write(f"OUTP{channel} OFF")

def configureSignal(inst, f: int, amplitude: float, offset:float, function_type: str = "SIN", channel: int = 1):
  base = f"SOUR{channel}"

  inst.write(f"{base}:FUNC {function_type}; FREQ {int(f)}; VOLT {amplitude}")
  inst.write(f"{base}:VOLT:OFFS {offset}")

def enableModulation(inst, type: str, m: float, f: int, function_type: str = "SIN", channel: int = 1):
  base = f"SOUR{channel}:{type}"
  inst.write(f"{base}:STAT ON; INT:FREQ {f}")

  if type == "AM":
    inst.write(f"{base}:DEPT {int(m * 100)}")
    print(int(m*100))
  elif type == "FM":
    inst.write(f"{base}:DEV {int(m)}")

def disableModulation(inst, type: str, channel: int = 1):
  inst.write(f"SOUR{channel}:{type}:STAT OFF")

In [9]:
rm = visa.ResourceManager("@py")
resources = rm.list_resources()
resources

('USB0::10893::36097::CN61150030::0::INSTR',)

In [88]:
wav_gen_id = resources[0]
wav_gen = rm.open_resource(wav_gen_id)

## Set Carrier Signal

In [137]:
configureSignal(wav_gen, f_c, 3, 2.5)

# Measure

In [ ]:
modulation_type="AM"
angle=0

In [ ]:
disableAWG(wav_gen)

rec = []
for f in vf:
  sd.rec(out=rec)
  sleep(0.1)
  enableModulation(wav_gen, modulation_type, 0.8, f)
  enableAWG()
  sleep(1)
  disableAWG()
  with wav.open(f"meas_{modulation_type}_{f*1e-3}kHz_{angle}deg.wav", mode="wb") as f:
    f.writeframes(rec) 
  
  